In [ ]:
import os, random, glob, pickle, collections, math
import numpy as np
import pandas as pd
import ujson as json
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
K.set_image_dim_ordering('tf')

In [ ]:
TRAIN_DIR = '../data/train/'
TEST_DIR = '../RFCN/JPEGImages/'
CHECKPOINT_DIR = './checkpoints/checkpoint4/'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
CONF_THRESH = 0.8
# like RCNN expand the crop by 16 pixels
p=16
ROWS = 224
COLS = 224
BatchSize = 128
LearningRate = 1e-4

In [ ]:
# GTbbox_df = ['image_class','image_file','crop_index','crop_class','xmin',''ymin','xmax','ymax']

if os.path.exists('../data/GTbbox_df.pickle'):
    print ('Loading from file GTbbox_df.pickle.')
    GTbbox_df = pd.read_pickle('../data/GTbbox_df.pickle')
else:
    print ('Generating file GTbbox_df.pickle.'.format(ROWS, COLS))       
    GTbbox_df = pd.DataFrame(columns=['image_class','image_file','crop_index','crop_class','xmin','ymin','xmax','ymax'])  

    crop_classes=FISH_CLASSES[:]
    crop_classes.remove('NoF')

    for c in crop_classes:
        print(c)
        j = json.load(open('../data/BBannotations/{}.json'.format(c), 'r'))
        for l in j: 
            filename = l["filename"]
            head, tail = os.path.split(filename)
            basename, file_extension = os.path.splitext(tail) 
            image = Image.open(TRAIN_DIR+c+'/'+tail)
            width_image, height_image = image.size
            for i in range(len(l["annotations"])):
                a = l["annotations"][i]
                xmin = (a["x"])
                ymin = (a["y"])
                width = (a["width"])
                height = (a["height"])
                delta_width = p/(COLS-2*p)*width
                delta_height = p/(ROWS-2*p)*height
                xmin_expand = xmin-delta_width
                ymin_expand = ymin-delta_height
                xmax_expand = xmin+width+delta_width
                ymax_expand = ymin+height+delta_height
                assert max(xmin_expand,0)<min(xmax_expand,width_image)
                assert max(ymin_expand,0)<min(ymax_expand,height_image)
                GTbbox_df.loc[len(GTbbox_df)]=[c,tail,i,a["class"],max(xmin_expand,0),max(ymin_expand,0),min(xmax_expand,width_image),min(ymax_expand,height_image)]
                if a["class"] != c: print(GTbbox_df.tail(1))


    #crop NoF by detections_full_AGNOSTICnms.pkl

    RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'

    with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    train_detections_full_AGNOSTICnms = detections_full_AGNOSTICnms[1000:]
    with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
        train_files = f.readlines()
    assert len(train_detections_full_AGNOSTICnms) == len(train_files)


    num_NoF_perIm = 10

    for im in range(len(train_detections_full_AGNOSTICnms)):
        if im%1000 == 0: print(im)

        basename = train_files[im][:9]
        image_class = train_files[im][10:-1]
        image = Image.open(TRAIN_DIR+image_class+'/'+basename+'.jpg')
        width_image, height_image = image.size

        bboxes = []
        detects_im = train_detections_full_AGNOSTICnms[im]
        for i in range(len(detects_im)):
            if detects_im[i,4] >= 0.999 and detects_im[i,2]<width_image and detects_im[i,3]<height_image:
                bboxes.append(detects_im[i,:]) 
        bboxes = np.asarray(bboxes)
        bboxes = bboxes[np.random.choice(bboxes.shape[0], num_NoF_perIm, replace=False), :]

        for j in range(len(bboxes)):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            width = xmax-xmin
            height = ymax-ymin
            delta_width = p/(COLS-2*p)*width
            delta_height = p/(ROWS-2*p)*height
            xmin_expand = xmin-delta_width
            ymin_expand = ymin-delta_height
            xmax_expand = xmax+delta_width
            ymax_expand = ymax+delta_height
            assert max(xmin_expand,0)<min(xmax_expand,width_image)
            assert max(ymin_expand,0)<min(ymax_expand,height_image)
            GTbbox_df.loc[len(GTbbox_df)]=[image_class,basename+'.jpg',j,'NoF',max(xmin_expand,0),max(ymin_expand,0),min(xmax_expand,width_image),min(ymax_expand,height_image)]

    GTbbox_df.to_pickle('../data/GTbbox_df.pickle')    
    
train_df, valid_df = train_test_split(GTbbox_df, test_size = 0.2, random_state=1986, stratify=GTbbox_df['crop_class'])

In [ ]:
#prepare for traing and validation data
def load_img(path, bbox, target_size=None):
    img = Image.open(path)
    img = img.convert('RGB')
    cropped = img.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
    if target_size:
        cropped = cropped.resize((target_size[1], target_size[0]))
    return cropped

def preprocess_input(x):
    #resnet50 image preprocessing
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

nb_perClass = int(BatchSize / len(FISH_CLASSES))   
samples_per_epoch=BatchSize*math.ceil(train_df.groupby('crop_class').size()['ALB']/nb_perClass)
def generator(datagen, df):
    while 1:
        batch_x = np.zeros((BatchSize, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BatchSize, len(FISH_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_perClass, replace=False),:]
        batch_df = df.groupby('crop_class', as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = os.path.join(row[0], row[1])
            fish = row[3]
            bbox = row[4:8]
            cropped = load_img(TRAIN_DIR+image_file,bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,FISH_CLASSES.index(fish)] = 1
            i += 1
        yield (batch_x, batch_y)

train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)
generator(datagen=train_datagen, df=train_df)


# validation_data (valid_x,valid_y)
df_1 = valid_df[valid_df.crop_class != 'NoF']
l = valid_df.groupby('crop_class').size()
l.pop('NoF')
nb_NoF_valid = math.ceil(l.sum()/10)
df_2 = valid_df[valid_df.crop_class == 'NoF'].sample(n=nb_NoF_valid)
valid_df = pd.concat([df_1,df_2], axis=0)
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    row = row.tolist()
    image_file = os.path.join(row[0], row[1])
    fish = row[3]
    bbox = row[4:8]
    cropped = load_img(TRAIN_DIR+image_file,bbox,target_size=(ROWS,COLS))
    x = np.asarray(cropped, dtype=K.floatx())
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i,FISH_CLASSES.index(fish)] = 1
    i += 1


In [ ]:
#callbacks

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        

model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        
learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)

tensorboard = TensorBoard(log_dir='./logs/log4', histogram_freq=0, write_graph=False, write_images=True)


In [ ]:
#Resnet50
#stg1 training

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
x = Dense(256, init='glorot_normal')(x)
x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), init='glorot_normal', activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=LearningRate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
samples_per_epoch=BatchSize*math.ceil(train_df.groupby('crop_class').size()['ALB']/nb_perClass)
model.fit_generator(generator(datagen=train_datagen, df=train_df), samples_per_epoch=samples_per_epoch, nb_epoch=30, verbose=1,
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(valid_x,valid_y), nb_worker=3, pickle_safe=True)

In [ ]:
#Resnet50
#stg4 training

files = glob.glob(CHECKPOINT_DIR+'*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
model = load_model(files[index])
# print('Loading model from weights.004-0.0565.hdf5')
# model = load_model('./checkpoints/checkpoint3/weights.004-0.0565.hdf5')

#164,142,80,38
for layer in model.layers[:38]:
   layer.trainable = False
for layer in model.layers[38:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(generator(datagen=train_datagen, df=train_df), samples_per_epoch=samples_per_epoch, nb_epoch=30, verbose=1,
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(valid_x,valid_y), nb_worker=3, pickle_safe=True)

In [ ]:
#GTbbox_CROPpred_df = GTbbox_df + ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

# test_data (test_x,test_y)
test_x = np.zeros((GTbbox_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
test_y = np.zeros((GTbbox_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
i = 0
for index,row in GTbbox_df.iterrows():
    row = row.tolist()
    image_file = os.path.join(row[0], row[1])
    bbox = row[4:8]
    cropped = load_img(TRAIN_DIR+image_file,bbox,target_size=(ROWS,COLS))
    x = np.asarray(cropped, dtype=K.floatx())
    x = preprocess_input(x)
    test_x[i] = x
    i += 1

files = glob.glob(CHECKPOINT_DIR+'*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
model = load_model(files[index])
# print('Loading model from weights.004-0.0565.hdf5')
# model = load_model('./checkpoints/checkpoint2/weights.004-0.0565.hdf5')

model.predict(x, batch_size=32, verbose=0)

In [ ]:
# RFCNbbox_RFCNpreds_df = ['image_file','crop_index','x_min','y_min','x_max','ymax',
#                          'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                          'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']
if os.path.exists('../data/RFCNbbox_RFCNpreds_df.pickle'):
    print ('Loading from file RFCNbbox_RFCNpreds_df.pickle.')
    RFCNbbox_RFCNpreds_df = pd.read_pickle('../data/RFCNbbox_RFCNpreds_df.pickle')
else:
    print ('Generating file RFCNbbox_RFCNpreds_df.pickle.'.format(ROWS, COLS))        
    RFCNbbox_RFCNpreds_df = pd.DataFrame(columns=['image_file','crop_index','x_min','y_min','x_max','ymax',
                                                  'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
                                                  'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']) 

    #crop by detections_full_AGNOSTICnms.pkl
    RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
    with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    assert len(detections_full_AGNOSTICnms) == len(test_files)

    for im in range(len(detections_full_AGNOSTICnms)):
        if im%1000 == 0: print(im)
        basename = test_files[im][:9]
        image = Image.open(TEST_DIR+'/'+basename+'.jpg')
        width_image, height_image = image.size
        
        bboxes = []
        detects_im = detections_full_AGNOSTICnms[im]
        for i in range(len(detects_im)):
            if np.max(detects_im[i,5:]) >= CONF_THRESH:
                bboxes.append(detects_im[i,:]) 
        if len(bboxes) == 0:
            ind = np.argmax(np.max(detects_im[:,5:], axis=1))
            bboxes.append(detects_im[ind,:])
        bboxes = np.asarray(bboxes)

        for j in range(len(bboxes)):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            width = xmax-xmin
            height = ymax-ymin
            delta_width = p/(COLS-2*p)*width
            delta_height = p/(ROWS-2*p)*height
            xmin_expand = xmin-delta_width
            ymin_expand = ymin-delta_height
            xmax_expand = xmax+delta_width
            ymax_expand = ymax+delta_height
            assert max(xmin_expand,0)<min(xmax_expand,width_image)
            assert max(ymin_expand,0)<min(ymax_expand,height_image)
            RFCNbbox_RFCNpreds_df.loc[len(RFCNbbox_RFCNpreds_df)]=[basename+'.jpg',j,max(xmin_expand,0),max(ymin_expand,0),
                                                                   min(xmax_expand,width_image),min(ymax_expand,height_image),
                                                                   bbox[4],bbox[5],bbox[6],bbox[7],bbox[8],bbox[9],bbox[10],bbox[11]]

    RFCNbbox_RFCNpreds_df.to_pickle('../data/RFCNbbox_RFCNpreds_df.pickle')      

In [ ]:
# RFCNbbox_CROPpreds_df = ['image_file','crop_index','x_min','y_min','x_max','ymax',
#                          'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                          'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']

files = glob.glob(CHECKPOINT_DIR+'*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
model = load_model(files[index])
# print('Loading model from weights.004-0.0565.hdf5')
# model = load_model('./checkpoints/checkpoint2/weights.004-0.0565.hdf5')

X_test_crop_centered = featurewise_center(X_test_crop)
test_crop_preds = model.predict(X_test_crop_centered, batch_size=BatchSize, verbose=1)

columns = ['ALB_BBCROP', 'BET_BBCROP', 'DOL_BBCROP', 'LAG_BBCROP', 'NoF_BBCROP', 'OTHER_BBCROP', 'SHARK_BBCROP', 'YFT_BBCROP']
BBCROP_preds_df = pd.DataFrame(test_crop_preds, columns=columns)

test_crop_files_BBCROP = test_crop_files
BBCROP_preds_df.insert(0, 'test_crop_files', test_crop_files_BBCROP)